# Data Exploration
- This notebook performs exploratory data analysis on the dataset.
- To expand on the analysis, attach this notebook to a cluster with runtime version **12.2.x-cpu-ml-scala2.12**,
edit [the options of pandas-profiling](https://pandas-profiling.ydata.ai/docs/master/rtd/pages/advanced_usage.html), and rerun it.
- Explore completed trials in the [MLflow experiment](#mlflow/experiments/871067855356899).

In [0]:
import mlflow
import os
import uuid
import shutil
import pandas as pd
import databricks.automl_runtime

# Download input data from mlflow into a pandas DataFrame
# Create temporary directory to download data
temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], "tmp", str(uuid.uuid4())[:8])
os.makedirs(temp_dir)

# Download the artifact and read it
training_data_path = mlflow.artifacts.download_artifacts(run_id="a9c416955dd343d8bf84f8aeabaa9c2f", artifact_path="data", dst_path=temp_dir)
df = pd.read_parquet(os.path.join(training_data_path, "training_data"))

# Delete the temporary data
shutil.rmtree(temp_dir)

target_col = "SalePrice"

# Drop columns created by AutoML before pandas-profiling
df = df.drop(['_automl_split_col_7282'], axis=1)

# Convert columns detected to be of semantic type datetime
datetime_columns = ["GarageYrBlt"]
df[datetime_columns] = df[datetime_columns].apply(pd.to_datetime, errors="coerce")

# Convert columns detected to be of semantic type numeric
numeric_columns = ["MasVnrArea"]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors="coerce")

## Semantic Type Detection Alerts

For details about the definition of the semantic types and how to override the detection, see
[Databricks documentation on semantic type detection](https://docs.microsoft.com/azure/databricks/applications/machine-learning/automl#semantic-type-detection).

- Semantic type `categorical` detected for columns `BedroomAbvGr`, `BsmtFullBath`, `BsmtHalfBath`, `Fireplaces`, `FullBath`, `GarageCars`, `HalfBath`, `KitchenAbvGr`, `MoSold`, `OverallCond`, `OverallQual`, `PoolArea`, `TotRmsAbvGrd`, `YrSold`. Training notebooks will encode features based on categorical transformations.
- Semantic type `datetime` detected for column `GarageYrBlt`. Training notebooks will convert each column to a datetime type and encode features based on temporal transformations.
- Semantic type `numeric` detected for column `MasVnrArea`. Training notebooks will convert each column to a numeric type and encode features based on numerical transformations.

## Profiling Results

In [0]:
from pandas_profiling import ProfileReport
df_profile = ProfileReport(df, minimal=True, title="Profiling Report", progress_bar=False, infer_dtypes=False)
profile_html = df_profile.to_html()

displayHTML(profile_html)

Number of variables,81
Number of observations,1460
Missing cells,89
Missing cells (%),0.1%
Total size in memory,924.0 KiB
Average record size in memory,648.1 B
Numeric,36
Categorical,44
DateTime,1
LotFrontage has a high cardinality: 111 distinct values,High cardinality
MSZoning is highly imbalanced (56.9%),Imbalance
